In [8]:
from azureml.core import Workspace, Datastore
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Dataset


In [3]:
subscription_id = "0a94de80-6d3b-49f2-b3e9-ec5818862801"
resource_group = "buas-y2"
workspace_name = "Test"

In [45]:
auth = InteractiveLoginAuthentication()
workspace = Workspace(subscription_id=subscription_id,
                   resource_group=resource_group,
                   workspace_name=workspace_name,
                   auth=auth,
                   )

In [17]:
# list all datastores registered in the current workspace
datastores = workspace.datastores
for name, datastore in datastores.items():
    print(name, datastore.datastore_type)

workspaceartifactstore AzureBlob
workspaceblobstore AzureBlob
workspaceworkingdirectory AzureFile


In [18]:
# Create a datastore object from the existing datastore named "workspaceblobstore".
datastore = Datastore(workspace, name='workspaceblobstore')

# Upload the data to the path target_path in datastore
datastore.upload(src_dir='test_data', target_path='transcripts', overwrite=True, show_progress=True)

Uploading an estimated of 14 files
Uploading test_data\train\chunk_01_segments.csv
Uploaded test_data\train\chunk_01_segments.csv, 1 files out of an estimated total of 14
Uploading test_data\train\chunk_03_segments.csv
Uploaded test_data\train\chunk_03_segments.csv, 2 files out of an estimated total of 14
Uploading test_data\train\chunk_05_segments.csv
Uploaded test_data\train\chunk_05_segments.csv, 3 files out of an estimated total of 14
Uploading test_data\train\chunk_07_segments.csv
Uploaded test_data\train\chunk_07_segments.csv, 4 files out of an estimated total of 14
Uploading test_data\train\chunk_08_segments.csv
Uploaded test_data\train\chunk_08_segments.csv, 5 files out of an estimated total of 14
Uploading test_data\validate\chunk_14_segments.csv
Uploaded test_data\validate\chunk_14_segments.csv, 6 files out of an estimated total of 14
Uploading test_data\test\chunk_11_segments.csv
Uploaded test_data\test\chunk_11_segments.csv, 7 files out of an estimated total of 14
Uploading

$AZUREML_DATAREFERENCE_67d7459c65ae4e0192a63805daba6ae3

# This is nice with SDKV1 but doesnt play nice with NEMO. The SDK2 way is way too complicated

In [16]:
# Create a FileDataset from a path to a directory.
# The directory contains a folder per class, each of which contains image files.
sample_set = Dataset.File.from_files(path=(datastore, 'transcripts'))
paths = sample_set.take_sample(0.001).take(30).download()
print(paths)

2025-05-25 13:51:19.189271 | ActivityCompleted: Activity=from_files, HowEnded=Failure, Duration=0.0 [ms], Info = {'activity_id': 'e1620620-bfd9-456b-b432-80bdc0dd4daa', 'activity_name': 'from_files', 'activity_type': 'PublicApi', 'app_name': 'FileDataset', 'source': 'azureml.dataset', 'version': '1.60.0', 'dataprepVersion': '', 'sparkVersion': '', 'subscription': '', 'run_id': '', 'resource_group': '', 'workspace_name': '', 'experiment_id': '', 'location': ''}, Exception=ImportError; Missing required package "azureml-dataset-runtime", which can be installed by running: "c:\Users\deanv\Dropbox\0_Buas\2024-2025\Y2D\Demo Project\mlops-demo-project\backend-microservice\.venv\Scripts\python.exe" -m pip install azureml-dataset-runtime --upgrade


ImportError: Missing required package "azureml-dataset-runtime", which can be installed by running: "c:\Users\deanv\Dropbox\0_Buas\2024-2025\Y2D\Demo Project\mlops-demo-project\backend-microservice\.venv\Scripts\python.exe" -m pip install azureml-dataset-runtime --upgrade

In [20]:
# Create a FileDataset from a path to a directory for the training data.
train_set = Dataset.File.from_files(path=(datastore, 'transcripts/train'))
# Split the dataset into train and validation sets
# train_set, val_set = train_set.random_split(0.8, seed=123)
val_set = Dataset.File.from_files(path=(datastore, 'transcripts/train'))
# Create a FileDataset from a path to a directory for the test data.
test_set = Dataset.File.from_files(path=(datastore, 'transcripts/test'))

#register the datasets
train_reg = train_set.register(workspace=workspace, name='transcripts_train', description='training data', create_new_version=True)
val_reg = val_set.register(workspace=workspace, name='transcripts_val', description='validation data', create_new_version=True)
test_reg = test_set.register(workspace=workspace, name='transcriptss_test', description='test data', create_new_version=True)

2025-05-25 14:02:55.082611 | ActivityCompleted: Activity=from_files, HowEnded=Failure, Duration=0.0 [ms], Info = {'activity_id': '444fff19-5219-4594-a6d7-a8cf31e2a0bc', 'activity_name': 'from_files', 'activity_type': 'PublicApi', 'app_name': 'FileDataset', 'source': 'azureml.dataset', 'version': '1.60.0', 'dataprepVersion': '', 'sparkVersion': '', 'subscription': '', 'run_id': '', 'resource_group': '', 'workspace_name': '', 'experiment_id': '', 'location': ''}, Exception=ImportError; Missing required package "azureml-dataset-runtime", which can be installed by running: "c:\Users\deanv\Dropbox\0_Buas\2024-2025\Y2D\Demo Project\mlops-demo-project\backend-microservice\.venv\Scripts\python.exe" -m pip install azureml-dataset-runtime --upgrade


ImportError: Missing required package "azureml-dataset-runtime", which can be installed by running: "c:\Users\deanv\Dropbox\0_Buas\2024-2025\Y2D\Demo Project\mlops-demo-project\backend-microservice\.venv\Scripts\python.exe" -m pip install azureml-dataset-runtime --upgrade

In [28]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import InteractiveBrowserCredential
import random

In [ ]:
# Initialize client
ml_client = MLClient(
    credential=InteractiveBrowserCredential(),
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name
)


In [36]:
# Get datastore reference
datastore = ml_client.datastores.get("workspaceblobstore")  # or your datastore name

In [ ]:
# Create data URIs (equivalent to Dataset.File.from_files)
train_uri = f"azureml://datastores/{datastore.name}/paths/transcript/train"
val_uri = f"azureml://datastores/{datastore.name}/paths/transcript/validaion"
test_uri = f"azureml://datastores/{datastore.name}/paths/transcript/test"

In [40]:
train_data = Data(
    name="transcripts_train",
    description="training data",
    path=train_uri,
    type=AssetTypes.URI_FOLDER
)

val_data = Data(
    name="transcripts_val",
    description="validation data",
    path=val_uri,
    type=AssetTypes.URI_FOLDER
)

test_data = Data(
    name="transcripts_test",
    description="test data", 
    path=test_uri,
    type=AssetTypes.URI_FOLDER
)

# Register the datasets
train_reg = ml_client.data.create_or_update(train_data)
val_reg = ml_client.data.create_or_update(val_data)
test_reg = ml_client.data.create_or_update(test_data)

print(f"Registered datasets:")
print(f"Train: {train_reg.name} v{train_reg.version}")
print(f"Val: {val_reg.name} v{val_reg.version}")
print(f"Test: {test_reg.name} v{test_reg.version}")

Registered datasets:
Train: transcripts_train v1
Val: transcripts_val v1
Test: transcripts_test v1


In [41]:
# list all datasets registered in the current workspace
datasets = workspace.datasets
for name, dataset in datasets.items():
    print(name)

In [43]:
# List all data assets
print("All registered data assets:")
print("-" * 50)

for data_asset in ml_client.data.list():
    print(f"📊 {data_asset.name}")
    print(f"   Version: {data_asset.version}")
    print(f"   Type: {data_asset.type}")
    print(f"   Description: {data_asset.description}")
    print(f"   Path: {data_asset.path}")
    print(f"   Created: {data_asset.creation_context.created_at}")
    print()

All registered data assets:
--------------------------------------------------
📊 transcripts_train
   Version: None
   Type: uri_folder
   Description: None
   Path: None
   Created: 2025-05-25 14:22:06.376834+00:00

📊 transcripts_val
   Version: None
   Type: uri_folder
   Description: None
   Path: None
   Created: 2025-05-25 14:22:07.623089+00:00

📊 transcripts_test
   Version: None
   Type: uri_folder
   Description: None
   Path: None
   Created: 2025-05-25 14:22:08.531985+00:00

